In [105]:
def plot_focal_plane():
    '''
    Plots the focal plane with the 5000 fiber positioners and 10 GFAs
    using the initial four corners of the 0th indexed GFA and rotating the
    points by 36 degrees counter-clockwise. 
    '''
    %pylab
    import desimodel.io
    # Sets the title of the graph
    title('Focal Plane Overhead View')
    # Plots the location of each of the fiber positioners
    fp = desimodel.io.load_fiberpos()
    plot(fp['X'],fp['Y'], 'g.')
    #x = [318.529, 330.901, 348.947, 336.574]
    #y = [225.702, 234.691, 209.830, 200.841]
    # Experiments with "Reference projection of active area" coordinates
    #x = [318.703, 331.075, 349.121, 336.748]
    #y = [225.816, 234.805, 209.944, 200.955]

    """ Uses the x and y from the petal indexed at 9 so the first petal 
        added to the table is indexed at 0
    [[ 313.24842144 -233.32358331]
     [ 325.62062672 -242.31230077]
     [ 307.55293135 -267.15753816]
     [ 295.18041705 -258.16786964]]"""
    x = [313.24842144, 325.62062672, 307.55293135, 295.18041705]
    y = [-233.32358331, -242.31230077, -267.15753816, -258.16786964]

    # Creates a rotation matrix for 36 degrees counter-clockwise
    rotateMat = numpy.zeros(shape=(2,2))
    rotateMat[0] = [cos(36*pi/180), -sin(36*pi/180)]
    rotateMat[1] = [sin(36*pi/180), cos(36*pi/180)]
    draw_gfas(x, y, rotateMat)
    
# Function that draws the GFAs on the focal plane
def draw_gfas(x, y, rotateMat):
    """
    Draws the 10 GFAs given the initial x and y coordinates of four corners of a GFA
    Parameters
    ----------
    x : Array of four x initial coordinates of the GFA
    y : Array of four y initial coordinates of the GFA
    """
    coord = numpy.zeros(shape=(2,1))
    gfaCoord = numpy.zeros(shape=(4, 2))
    oldXCoord = x
    oldYCoord = y
    for j in range(10):
        for i in range(4):
            coord[0] = oldXCoord[i]
            coord[1] = oldYCoord[i]
            newCoord = matmul(rotateMat, coord)
            oldXCoord[i] = newCoord[0]
            oldYCoord[i] = newCoord[1]
            gfaCoord[i] = [newCoord[0], newCoord[1]]
            plot(newCoord[0], newCoord[1], 'k.')
        draw_single_gfa(gfaCoord)
            
def draw_single_gfa(gfaCoord):
    """
    Draws a single GFA given a 4X2 array of coordinates for the four corners of a GFA
    Parameters
    ----------
    gfaCoord: 4X2 array of x and y coordinates with each row representing a corner of the GFA
    """
    # Prints all of the GFA coordinates for debugging
    #print(gfaCoord) 
    gfaPolygon = Polygon(gfaCoord)
    plt.gca().add_patch(gfaPolygon)


In [106]:
plot_focal_plane()


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [107]:
def build_gfa_table():
    '''
    Builds the GFA table given the data from DESI-0530-v13 Excel spreadsheet
    and writes a .ecsv file using the astropy table library.
    '''
    # Uses the reference projection of active area to create data table of GFAs
    from astropy.table import Table
    # Initial x and y coordinates for the GFAs
    """ Uses the x and y from the petal indexed at 9 so the first petal 
        added to the table is indexed at 0
    [[-125.10482863 -370.01790486]
     [-129.83038525 -384.56223777]
     [-159.04283509 -375.05643893]
     [-154.31646944 -360.51151824]]   
    """
    # Data obtained from DESI-0530-v13 Excel spreadsheet
    x = [-125.10482863, -129.83038525, -159.04283509, -154.31646944]
    y = [-370.01790486, -384.56223777, -375.05643893, -360.51151824]
    z = [-17.053, -18.487, -18.631, -17.198]
    rotateMat = numpy.zeros(shape=(2,2))
    rotateMat[0] = [cos(36*pi/180), -sin(36*pi/180)]
    rotateMat[1] = [sin(36*pi/180), cos(36*pi/180)]
    
    # Note: the corners are 0 indexed 
    gfaTable = Table(names = ('PETAL', 'CORNER', 'X', 'Y', 'Z'), 
                 dtype = ('int', 'int', 'float', 'float', 'float'))
    # Sets the units for the GFA table
    gfaTable['X'].unit = 'mm'
    gfaTable['Y'].unit = 'mm'
    gfaTable['Z'].unit = 'mm'
    find_gfa_coordinates(x, y, gfaTable, rotateMat)
    # Saves the table of data as an ecsv file
    gfaTable.write('gfa.ecsv', format='ascii.ecsv')
    
# Function that obtains the x and y coordinates for each corner of the GFAs
def find_gfa_coordinates(x, y, gfaTable, rotateMat):
    '''
    Finds all the GFA coordinates by rotating the initial coordinates and adding
    the respective coordinates to the gfaTable
    Parameters
    ----------
    x : Array of four x initial coordinates of the GFA
    y : Array of four y initial coordinates of the GFA
    gfaTable: Astropy Table object which stores the petal number, corner number, 
    and x, y, and z coordinates in mm within each row
    '''
    coord = numpy.zeros(shape=(2,1))
    GFAcoord = numpy.zeros(shape=(4, 2))
    oldXCoord = x
    oldYCoord = y
    for j in range(10):
        for i in range(4):
            coord[0] = oldXCoord[i]
            coord[1] = oldYCoord[i]
            newCoord = matmul(rotateMat, coord)
            oldXCoord[i] = newCoord[0]
            oldYCoord[i] = newCoord[1]
            GFAcoord[i] = [newCoord[0], newCoord[1]]
            # Could be building the table in O(N^2), which is notably inefficient
            gfaTable.add_row([j, i, newCoord[0], newCoord[1], z[i]])

In [108]:
build_gfa_table()